In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
item_categories = pd.read_csv(r"C:\Users\HP\Documents\DATASCIENCE\competitive-data-science-predict-future-sales/item_categories.csv", sep=",")
items = pd.read_csv(r"C:\Users\HP\Documents\DATASCIENCE\competitive-data-science-predict-future-sales/items.csv", sep=",")
shops = pd.read_csv(r"C:\Users\HP\Documents\DATASCIENCE\competitive-data-science-predict-future-sales/shops.csv", sep=",")
test = pd.read_csv(r"C:\Users\HP\Documents\DATASCIENCE\competitive-data-science-predict-future-sales/test.csv", sep=",")
sales_train = pd.read_csv(r"C:\Users\HP\Documents\DATASCIENCE\competitive-data-science-predict-future-sales/sales_train.csv", sep=",")

In [11]:
# Import Libraries
class pipeline1():
    def __init__(self,train,test):
        self.train = train
        self.test = test
        
    def preprocessdata(self):
        import pandas as pd
        import matplotlib.pyplot as plt
        import numpy as np
        import seaborn as sns
        pd.options.display.max_columns = 1000
        import warnings
        warnings.filterwarnings(action='ignore',category=DeprecationWarning)
        warnings.filterwarnings(action='ignore',category=FutureWarning)
        from sklearn.model_selection import train_test_split
        import xgboost as xgb
        %matplotlib inline
        
        itemid = list(self.train.item_id)
        price = list(self.train.item_price)
        shopid = list(self.train.shop_id)

        itemidt = list(self.test.item_id)
        shopidt = list(self.test.shop_id)

        ve = []
        for i in range(len(self.train)):
            ve.append(str(shopid[i])+str(itemid[i]))

        self.train['ve'] = ve
        voo = list(self.train['ve'].unique())

        vet = []
        for i in range(len(test)):
            vet.append(str(shopidt[i])+str(itemidt[i]))
        self.test['ve'] = vet

        self.train = sales_train.drop(['item_price','date'],axis = 1)
        
        p = [int(i) for i in (list(self.test['ve']))]
        v = [int(i) for i in (list(self.train['ve']))]
        self.train['ve'] = v
        self.test['ve'] = p
        self.train['ve'] = self.train['ve']/1200
        self.test['ve'] = self.test['ve']/1200
    
    def seperatetofeatures(self):
        traini = self.train[:2500000]
        validation = self.train[2500000:]
        Xval = validation.drop(['item_cnt_day','date_block_num'],axis = 1)
        yval = validation.item_cnt_day
        X = traini.drop(['date_block_num','item_cnt_day'], axis = 1)
        y = traini.item_cnt_day
        self.X = X
        self.Xval = Xval
        self.yval = yval
        self.y = y
        
    def predictlinearreg(self):
        from sklearn.metrics import mean_squared_error
        from sklearn import linear_model
        X_train, X_test, y_train, y_test = train_test_split(self.X,self.y, test_size=0.20)
        reg = linear_model.LinearRegression()
        reg.fit(X_train,y_train)
        predicted_scoretrainreg = reg.predict(X_train)
        predicted_scoretestreg = reg.predict(X_test)
        predicted_scorevalreg = reg.predict(self.Xval)
        self.reg = reg
        self.y_train = y_train
        self.y_test = y_test
        self.predicted_scoretrainreg = predicted_scoretrainreg
        self.predicted_scoretestreg = predicted_scoretestreg
        self.predicted_scorevalreg = predicted_scorevalreg
        
    
    def xgboost(self):
        clfg=xgb.XGBRegressor()
        X_train, X_test, y_train, y_test = train_test_split(self.X,self.y, test_size=0.20)
        clfg.fit(X_train,y_train)
        predicted_scoretrainxgb = clfg.predict(X_train)
        predicted_scoretestxgb = clfg.predict(X_test)
        predicted_scorevalxgb = clfg.predict(self.Xval)
        self.clfg = clfg
        self.predicted_scoretrainxgb = predicted_scoretrainxgb
        self.predicted_scoretestxgb = predicted_scoretestxgb
        self.predicted_scorevalxgb = predicted_scorevalxgb
        
        
    def finaltrain(self):
        from sklearn import linear_model
        from sklearn.metrics import mean_squared_error
        reg = linear_model.LinearRegression()
        prediction = pd.DataFrame()
        predictiont = pd.DataFrame()
        predictionv = pd.DataFrame()
        prediction['xgbtrain'] = self.predicted_scoretrainxgb
        prediction['regtrain'] = self.predicted_scoretrainreg
        predictiont['xgbtest'] = self.predicted_scoretestxgb
        predictiont['regtest'] = self.predicted_scoretestreg
        predictionv['xgbval'] = self.predicted_scorevalxgb
        predictionv['regval'] = self.predicted_scorevalreg
        reg.fit(prediction,self.y_train)
        pret = reg.predict(predictiont)
        prev = reg.predict(predictionv)
        self.reg2 = reg
        errort,errorv = mean_squared_error(self.y_test,pret), mean_squared_error(self.yval,prev)
    
    def seperatetofetest(self):
        submissionid = self.test['ID']
        self.test = self.test.drop(['ID'],axis = 1)
        testregpred = self.reg.predict(self.test)
        testxgbpred = self.clfg.predict(self.test)
        predictiontest = pd.DataFrame()
        predictiontest['xgbtrain'] = testxgbpred
        predictiontest['regtrain'] = testregpred
        final = self.reg2.predict(predictiontest)
        sub = pd.DataFrame()
        sub['ID'] = submissionid
        sub['item_cnt_month'] = final
        sub.to_csv('letsemova44.csv', index=False)
        return sub
        
    
        

In [12]:
p = pipeline1(sales_train,test)
p.preprocessdata()
p.seperatetofeatures()
p.predictlinearreg()
p.xgboost()
p.finaltrain()
g = p.seperatetofetest()

In [13]:
g

,ID,item_cnt_month
0,0,1.186670
1,1,1.191062
2,2,1.189894
3,3,1.189878
4,4,1.190446
...,...,...
214195,214195,1.276795
214196,214196,1.239568
214197,214197,1.232522
214198,214198,1.296395
